In [1]:
!pip install kaggle

In [2]:
!kaggle datasets download -d andrewmvd/sp-500-stocks -p data/market_data

Dataset URL: https://www.kaggle.com/datasets/andrewmvd/sp-500-stocks
License(s): CC0-1.0
100%|███████████████████████████████████████| 42.7M/42.7M [02:18<00:00, 375kB/s]
100%|███████████████████████████████████████| 42.7M/42.7M [02:18<00:00, 323kB/s]


In [4]:
!mv data/market_data/sp-500-stocks.zip ~/sp500.zip

In [4]:
import numpy as np
import pandas as pd

In [5]:
# models
from models import FamaFrench3Factor, FamaFrench5Factor, Carhart